Encode images using InceptionV3's pre-fc layer

------

In [1]:
import re
import os
import sys
import exceptions
import numpy as np
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

---

In [3]:
# get files in directory
def get_files(src_dir):
    f = []
    for (dirpath, dirnames, filenames) in os.walk(src_dir):
        f.extend(filenames)
        break
    return f

# create graph from pb file
def create_graph(pb_file, sess):
    with sess:        
        with tf.gfile.FastGFile(pb_file, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            tf.import_graph_def(graph_def, name='')
    return sess.graph

---

In [10]:
INPUT_DIRS = [
    '/datasets/kaggle/painters/train',
    '/datasets/kaggle/painters/test'
]

OUTPUT_DIRS = {
    'bottlenecks': 'out/bottlenecks',
    'dimensions': 'out/dimensions',
    'exifs': 'out/exifs',
}

In [5]:
for v in OUTPUT_DIRS.values():
    if not os.path.isdir(v): os.makedirs(v)

In [6]:
# init tf sess
sess = tf.Session()

# restore inceptionv3 graph
create_graph('classify_image_graph_def.pb', sess)

# summary writer for tensorboard
summary_writer = tf.train.SummaryWriter('/tmp/tf-painters-b', graph=sess.graph)

In [7]:
# find tensors we're interested in
bottneck_t = sess.graph.get_tensor_by_name('pool_3/_reshape:0')
decode_jpeg_t = sess.graph.get_tensor_by_name('DecodeJpeg/contents:0')

In [8]:
i = 0
files = []

for d in INPUT_DIRS:
    df = get_files(d)
    for f in df:
        files.append(d + '/' + f)

In [11]:
i = 0

for f in tqdm(files):
    
    i = i + 1

    if 0 == i % 100: summary_writer.flush()  

    # if corrected file exists, use it
    f_corrected = f + '.correct.jpg'
    if (os.path.isfile(f_corrected)): f = f_corrected
        
    # file id
    fid = int(re.findall(r"\d+", f)[0])
    
    f_bottneck = OUTPUT_DIRS['bottlenecks'] + '/' + str(fid)
    f_dims = OUTPUT_DIRS['dimensions'] + '/' + str(fid)
    f_exifs = OUTPUT_DIRS['exifs'] + '/' + str(fid)
    
    if os.path.isfile(f_bottneck) and \
        os.path.isfile(f_dims) and \
        os.path.isfile(f_exifs): continue
        
    try:
        
        img = Image.open(f)
        
        exif = img._getexif()
        dims = img.getbbox()[2:]

#         print f
        image_data = tf.gfile.FastGFile(f, 'rb').read()

        # run session with an image
        bottneck = sess.run(bottneck_t, {decode_jpeg_t: image_data})
        bottneck = np.squeeze(bottneck)
        
        # save data to files
        
        bottneck.tofile(f_bottneck)        
        np.array(dims).astype(np.float32).tofile(f_dims)
        
        with open(f_exifs, 'wb') as fh:
            fh.write(str(exif))
            fh.close()
            
    except:
        e = sys.exc_info()[0]
        if exceptions.KeyboardInterrupt == e: raise
        print '!', f, sys.exc_info()[0]

  8%|▊         | 8297/103507 [00:00<00:02, 38134.84it/s]

/datasets/kaggle/painters/train/79499.jpg.correct.jpg
/datasets/kaggle/painters/train/92899.jpg.correct.jpg


 20%|██        | 20806/103507 [00:00<00:01, 47725.79it/s]

/datasets/kaggle/painters/train/91033.jpg.correct.jpg
/datasets/kaggle/painters/train/3917.jpg.correct.jpg
/datasets/kaggle/painters/train/101947.jpg.correct.jpg


 34%|███▍      | 35145/103507 [00:00<00:02, 31609.45it/s]

...

/datasets/kaggle/painters/train/95347.jpg.correct.jpg


 83%|████████▎ | 85732/103507 [00:02<00:00, 48774.51it/s]

/datasets/kaggle/painters/test/20153.jpg.correct.jpg
/datasets/kaggle/painters/test/100532.jpg.correct.jpg


100%|██████████| 103507/103507 [00:02<00:00, 42707.44it/s]

/datasets/kaggle/painters/test/18649.jpg.correct.jpg
